In [82]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
import psycopg
import requests
import pandas as pd
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)
from src.paths import DATA_PATH
from src.database.connection import connect
from src.database.db_functions import get_amazon_dataframe

In [80]:
with connect(db_key="main") as conn:
    with conn.cursor() as cur:
        df = get_amazon_dataframe(cur=cur, categories=None, limit=10000)

In [83]:
marimekko_data_url = "http://127.0.0.1:8000/dashboard/all_categories/marimekko_price_volume"

try:
    marimekko_response = requests.post(marimekko_data_url)
    marimekko_response.raise_for_status()
    marimekko_data = marimekko_response.json()  # Expecting a JSON array of dicts
except requests.exceptions.RequestException as e:
    raise Exception(f"An error occurred fetching Marimekko data: {e}")

In [84]:
df_marimekko = pd.DataFrame(marimekko_data)

# Normalize the data for better visualization
df_marimekko['relative_price'] = df_marimekko['total_price'] / df_marimekko['total_price'].sum()
df_marimekko['relative_rating'] = df_marimekko['total_rating_number'] / df_marimekko['total_rating_number'].sum()

    main_category  total_price  total_rating_number  total_volume
0      All Beauty   4030873.50            243098956  4.477788e+09
1  AMAZON FASHION  11877148.00           1376382821  3.277100e+10
2  Premium Beauty    172200.03              4737515  2.141057e+08


In [85]:
df_marimekko

,main_category,total_price,total_rating_number,total_volume,relative_price,relative_rating
0,All Beauty,4030873.50,243098956,4.477788e+09,0.250673,0.149671
1,AMAZON FASHION,11877148.00,1376382821,3.277100e+10,0.738618,0.847412
2,Premium Beauty,172200.03,4737515,2.141057e+08,0.010709,0.002917


In [86]:
import plotly.graph_objects as go


marimekko_fig = go.Figure(
go.Treemap(
    labels=df_marimekko['main_category'],
    parents=[""] * len(df_marimekko),  # Treemap needs a parent-child relationship
    values=df_marimekko['total_volume'],  # Define the size of each rectangle
    customdata=df_marimekko[['total_price', 'total_rating_number']],
    texttemplate=(
        "<b>%{label}</b><br>"
        "Total Price: %{customdata[0]:,.2f}<br>"
        "Total Ratings: %{customdata[1]:,.0f}<br>"
    ),
    hovertemplate=(
        "<b>%{label}</b><br>"
        "Total Price: %{customdata[0]:,.2f}<br>"
        "Total Ratings: %{customdata[1]:,.0f}<br>"
    ),
    marker=dict(
        colors=df_marimekko['total_rating_number'],  # Color by total_rating_number
        colorscale='Blues',
    )
)
)

marimekko_fig.update_layout(
title="Marimekko Chart of Categories by Total Price and Ratings",
margin=dict(t=50, l=25, r=25, b=25)
)